In [1]:
import os
import platform
import sys

def get_project_root() -> str:
    system = platform.system()
    return r"D:\Users\tcshore\Documents\Projects\Tangrams\Data" if system == "Windows" else "/home/tshore/Projects/tangrams-restricted/Data"

infile_path = os.path.join(get_project_root(), "wordprobseqs-inflected-small.tsv")
print("Will read file \"{}\".".format(infile_path), file=sys.stderr)

Will read file "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv".


In [2]:
import csv
import pandas as pd

RESULTS_FILE_CSV_DIALECT = csv.excel_tab
# NOTE: "category" dtype doesn't work with pandas-0.21.0 but does with pandas-0.21.1
__RESULTS_FILE_DTYPES = {"DYAD": "category", "WORD": "category", "IS_TARGET": bool, "IS_OOV": bool,
				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}

def read_results_file(inpath: str, encoding: str) -> pd.DataFrame:
	print("Reading \"{}\" using encoding \"{}\".".format(inpath, encoding), file=sys.stderr)
	result = pd.read_csv(inpath, dialect=RESULTS_FILE_CSV_DIALECT, sep=RESULTS_FILE_CSV_DIALECT.delimiter,
						 float_precision="round_trip",
						 encoding=encoding, memory_map=True, dtype=__RESULTS_FILE_DTYPES)
	return result

cv_results = read_results_file(infile_path, "windows-1252")
print("Read {} cross-validation results for {} dyad(s).".format(cv_results.shape[0], cv_results["DYAD"].nunique()),
      file=sys.stderr)

Reading "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv" using encoding "windows-1252".
Read 180880 cross-validation results for 2 dyad(s).


In [3]:
def find_target_ref_rows(df: pd.DataFrame) -> pd.DataFrame:
	result = df.loc[df["IS_TARGET"] == True]
	result_row_count = result.shape[0]
	complement_row_count = df.loc[~df.index.isin(result.index)].shape[0]
	assert result_row_count + complement_row_count == df.shape[0]
	print("Found {} nontarget rows and {} target rows. Ratio: {}".format(complement_row_count, result_row_count,
																		 complement_row_count / float(
																			 result_row_count)), file=sys.stderr)
	return result

cv_results = find_target_ref_rows(cv_results)

Found 171836 nontarget rows and 9044 target rows. Ratio: 19.0


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#import numpy as np

# Create vocab before splitting training and testing DFs so that the word feature set is stable
#vocab_words = tuple(sorted(cv_results["WORD"].unique()))
print("Fitting one-hot encoder for vocabulary of size {}.".format(len(cv_results["WORD"].unique())), file=sys.stderr)

# https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
# integer encode
label_encoder = LabelEncoder()
vocab_labels = label_encoder.fit_transform(cv_results["WORD"])
cv_results["WORD_LABEL"] = vocab_labels
#print(vocab_labels)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
vocab_labels = vocab_labels.reshape(len(vocab_labels), 1)
onehot_encoder.fit(vocab_labels)
#assert onehot_encoder.n_values_ == len(vocab_words)
#vocab_onehot_encoded = onehot_encoder.fit_transform(vocab_labels)
#print(vocab_onehot_encoded)
# invert first example
#inverted = label_encoder.inverse_transform([np.argmax(vocab_onehot_encoded[0, :])])
#print(inverted)

Fitting one-hot encoder for vocabulary of size 299.


OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [5]:
from typing import Tuple

import numpy as np

def split_training_testing(df: pd.DataFrame, test_set_size : int) -> Tuple[pd.DataFrame, pd.DataFrame]:
	dyad_ids = df["DYAD"].unique()
	training_set_size = len(dyad_ids) - test_set_size
	if training_set_size < 1:
		raise ValueError("Desired test set size is {} but only {} dyads found.".format(test_set_size, len(dyad_ids)))
	else:
		training_set_dyads = frozenset(np.random.choice(dyad_ids, training_set_size))
		print("Training set dyads: {}".format(sorted(training_set_dyads)), file=sys.stderr)
		training_set_idxs = df["DYAD"].isin(training_set_dyads)
		training_set = df.loc[training_set_idxs]
		test_set = df.loc[~training_set_idxs]
		test_set_dyads = frozenset(test_set["DYAD"].unique())
		print("Test set dyads: {}".format(sorted(test_set_dyads)), file=sys.stderr)

		assert not frozenset(training_set["DYAD"].unique()).intersection(frozenset(test_set_dyads))
		return training_set, test_set
    
training_df, test_df = split_training_testing(cv_results, 1)
training_df = training_df.copy(deep=False)
test_df = test_df.copy(deep=False)

Training set dyads: ['1']
Test set dyads: ['17']


In [63]:
from typing import Iterator, List, Mapping

class SequenceMatrixFactory(object):

	def __init__(self, label_encoder, onehot_encoder):
		self.label_encoder = label_encoder
		self.onehot_encoder = onehot_encoder
    
	@property
	def feature_count(self) -> int:
		word_features = self.onehot_encoder.n_values_[0]
		return word_features + 3

	def __create_datapoint_feature_array(self, row: pd.Series) -> List[float]:
		#word_features = [0.0] * len(self.__vocab_idxs)
		# The features representing each individual vocabulary word are at the beginning of the feature vector
		#word_features[self.__vocab_idxs[row["WORD"]]] = 1.0
		#word_label = self.label_encoder.transform(row["WORD"])
		word_label = row["WORD_LABEL"]
		#print("Word label: {}".format(word_label), file=sys.stderr)
        # "OneHotEncoder.transform(..)" returns a matrix even if only a single value is passed to it, so get just the first (and only) row
		word_features = self.onehot_encoder.transform(word_label)[0]
		#print("Word features: {}".format(word_features), file=sys.stderr)
		# The word label for the one-hot encoding is that with the same index as the column that has a "1" value, i.e. the highest value in the vector of one-hot encoding values
		#inverse_label = np.argmax(word_features)
		#assert inverse_label == word_label
		#inverse_word = self.label_encoder.inverse_transform([inverse_label])
		#print("Inverse word label: {}".format(inverse_label), file=sys.stderr)
		is_instructor = 1.0 if row["IS_INSTRUCTOR"] else 0.0
		is_oov = 1.0 if row["IS_OOV"] else 0.0
		#is_target = 1.0 if row["IS_TARGET"] else 0.0
		score = row["PROBABILITY"]
		other_features = np.array((is_instructor, is_oov, score))
		#result = word_features + other_features
		result = np.concatenate((word_features, other_features))
		#print("Created a vector of {} features.".format(len(result)), file=sys.stderr)
		return result

	def __call__(self, df : pd.DataFrame) -> np.array:
		# https://stackoverflow.com/a/47815400/1391325
		df.sort_values("TOKEN_SEQ_ORDINALITY", inplace=True)
		sequence_groups = df.groupby(("CROSS_VALIDATION_ITER", "DYAD", "SPLIT_SEQ_NO", "UTT_START_TIME", "UTT_END_TIME", "ENTITY"), as_index=False)
		return np.array(tuple(tuple(self.__create_feature_vectors(seq)) for _, seq in sequence_groups))
        
	def __create_feature_vectors(self, df : pd.DataFrame) -> Iterator[List[float]]:
		# noinspection PyProtectedMember
		return (self.__create_datapoint_feature_array(row._asdict()) for row in df.itertuples(index=False))

print("Splitting token sequences.", file=sys.stderr)
seq_matrix_factory = SequenceMatrixFactory(label_encoder, onehot_encoder)
training_matrix = seq_matrix_factory(training_df)
print("Created a training data matrix of shape {}.".format(training_matrix.shape), file=sys.stderr)
test_matrix = seq_matrix_factory(test_df)
print("Created a test data matrix of shape {}.".format(test_matrix.shape), file=sys.stderr)

Splitting token sequences.
Created a training data matrix of shape (1087, 4, 302).
Created a test data matrix of shape (1174, 4, 302).


In [64]:
training_x = training_matrix[:,:,:-1]
print(training_x.shape)
assert len(training_x.shape) == 3
training_y = training_matrix[:,:,-1]
print(training_y.shape)
assert len(training_y.shape) == 2

from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

model = Sequential()
#word_embeddings = Embedding(len(vocab), embedding_vector_length, input_length=max_review_length)
#model.add(word_embeddings)
# model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# input shape is a pair of (timesteps, features) <https://stackoverflow.com/a/44583784/1391325>
input_shape = training_x.shape[1:]
print("Input shape: {}".format(input_shape), file=sys.stderr)
units = training_y.shape[1]
print("Units: {}".format(units), file=sys.stderr)
lstm = LSTM(input_shape=input_shape, units=units)
#lstm = LSTM(batch_input_shape = training_x.shape, stateful = True, units=len(training_y.shape))
model.add(lstm)
model.add(Dense(units, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

(1087, 4, 301)
(1087, 4)


Input shape: (4, 301)
Units: 4


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 4)                 4896      
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 20        
Total params: 4,916
Trainable params: 4,916
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
# train LSTM
#model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
training_history = model.fit(x=training_x, y=training_y, epochs=1, verbose=1)

# train LSTM <https://machinelearningmastery.com/memory-in-a-long-short-term-memory-network/>
#epochs = 5
#for i in range(epochs):
#    x = training_x[i]
#    x_oldshape = x.shape
#    x_newshape = (-1, x_oldshape[0], x_oldshape[1])
#    x = np.reshape(x, x_newshape)
#    #print("x.shape = {}".format(x.shape), file=sys.stderr)
#    y = training_y[i]
#    y_oldshape = y.shape
#   y_newshape = (-1, y_oldshape[0])
#    y = np.reshape(y, y_newshape)
#    #print("y.shape = {}".format(y.shape), file=sys.stderr)
#    model.fit(x, y, epochs=1, batch_size=1, verbose=1, shuffle=False)
#    model.reset_states()
    
    

Epoch 1/1
1087/1087 [==============================] - 1s 905us/step - loss: 0.6804 - acc: 0.2114


In [67]:
# test LSTM

def onehot_encodings(features : np.array, onehot_encoder) -> np.array:
    """
    :param features: A 2D array of feature vectors, each of which starts with one-hot encoding features. Any other features follow those.
    :param onehot_encoder: The instance used to encode the original values.
    :return: A 2D numpy array consisting only of one-hot encoding features.
    """
    feature_count = onehot_encoder.n_values_[0]
    return features[:,:feature_count + 1]


test_x = test_matrix[:,:,:-1]
print(test_x.shape)
assert len(test_x.shape) == 3
test_y = test_matrix[:,:,-1]
print(test_y.shape)
assert len(test_y.shape) == 2

seq_predicted_values = model.predict(test_x, verbose=0)
model.reset_states()
#print("result.shape = {}".format(seq_predicted_values.shape), file=sys.stderr)
for i, tested_seq in enumerate(test_x):
    word_features = onehot_encodings(tested_seq, onehot_encoder)
    #print("word features: {}".format(word_features))
    #print("Tested sequence: {}".format(tested_seq))
    #print("Tested sequence shape: {}".format(tested_seq.shape))
    seq_labels = word_features.argmax(axis=1)
    #print("Inverse labels: {}".format(inverse_labels))
    seq_words = label_encoder.inverse_transform(seq_labels)
    print("Inverse word labels: {}".format(seq_words))
    predicted_values = seq_predicted_values[i]
    assert tested_seq.shape[:-1] == predicted_values.shape
    #assert tested_seq.shape[:-1] == actual_values.shape
    #rint("Predicted values: {}".format(predicted_values))
    actual_values = test_y[i]
    assert predicted_values.shape == actual_values.shape
    
    differences = predicted_values - actual_values
    print("[Predicted_val - Actual_val]: {}".format(differences))
    #print("Actual values: {}".format(actual_values))
	#print('X=%.1f y=%.1f, yhat=%.1f' % (seq1[i], seq1[i+1], result[i]))

(1174, 4, 301)
(1174, 4)
feature_count: 299
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'so']
[Predicted_val - Actual_val]: [ 0.41309327  0.49665868  0.47341868  0.00859977]
feature_count: 299
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'okay']
[Predicted_val - Actual_val]: [ 0.40998071  0.49664134  0.46952748  0.05711366]
feature_count: 299
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'next']
[Predicted_val - Actual_val]: [ 0.41569224  0.49725792  0.47150332 -0.0022002 ]
feature_count: 299
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' '`']
[Predicted_val - Actual_val]: [ 0.42013109  0.48691452  0.46122789  0.07938372]
feature_count: 299
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'yeah']
[Predicted_val - Actual_val]: [ 0.41464955  0.49907237  0.47254509 -0.05413228]
feature_count: 299
Inverse word labels: ['__PADDING__' '__PADDING__' '__PADDING__' 'no']
[Predicted_val - Actual_val]: [ 0.40

[Predicted_val - Actual_val]: [ 0.43106315  0.02984559  0.09125833 -0.04557881]
feature_count: 299
Inverse word labels: ['__PADDING__' 'it' "'s" 'like']
[Predicted_val - Actual_val]: [ 0.44574192  0.03835907  0.00292049  0.16260285]
feature_count: 299
Inverse word labels: ['__PADDING__' 'ah' 'it' "'s"]
[Predicted_val - Actual_val]: [ 0.44656533 -0.10163797 -0.0869392  -0.05490738]
feature_count: 299
Inverse word labels: ['__PADDING__' 'it' 'is' 'pink']
[Predicted_val - Actual_val]: [ 0.44303149 -0.00991528 -0.10398715 -0.15618387]
feature_count: 299
Inverse word labels: ['__PADDING__' 'it' "'s" 'a']
[Predicted_val - Actual_val]: [ 0.43717745 -0.01743336 -0.05032846 -0.01077872]
feature_count: 299
Inverse word labels: ['__PADDING__' 'the' 'orange' 'one']
[Predicted_val - Actual_val]: [ 0.44502339 -0.02740232 -0.29965274  0.00844455]
feature_count: 299
Inverse word labels: ['__PADDING__' 'yeah' 'it' "'s"]
[Predicted_val - Actual_val]: [ 0.44428071 -0.04857738 -0.02892193  0.00541055]
fea

Inverse word labels: ['a' 'light' 'blue' 'piece']
[Predicted_val - Actual_val]: [-0.10512744 -0.33690849 -0.27162633 -0.16972257]
feature_count: 299
Inverse word labels: ['a' 'pink' 'purple' 'violet']
[Predicted_val - Actual_val]: [-0.01057291 -0.45511976 -0.3123814   0.50726766]
feature_count: 299
Inverse word labels: ['a' 'green' 'V' 'shape']
[Predicted_val - Actual_val]: [-0.1744908  -0.38771548 -0.44504387 -0.00636423]
feature_count: 299
Inverse word labels: ['one' 'triangle' 'on' 'top']
[Predicted_val - Actual_val]: [ 0.01485345 -0.11570186 -0.11777936  0.15138183]
feature_count: 299
Inverse word labels: ['a' 'small' 'blue' 'oh']
[Predicted_val - Actual_val]: [-0.06355387 -0.3225889  -0.28798537  0.11703587]
feature_count: 299
Inverse word labels: ['the' 'green' 'one' 'in']
[Predicted_val - Actual_val]: [-0.03144297 -0.09947964  0.03157833  0.02032498]
feature_count: 299
Inverse word labels: ['a' 'blue' 'in' 'the']
[Predicted_val - Actual_val]: [-0.00088233 -0.09631844  0.06574444

Inverse word labels: ['with' 'no' 'in' 'the']
[Predicted_val - Actual_val]: [ 0.10838266  0.13488557  0.01229245  0.01684649]
feature_count: 299
Inverse word labels: ['the' 'shape' 'of' 'person']
[Predicted_val - Actual_val]: [-0.05378142  0.22671062 -0.04446318 -0.48337533]
feature_count: 299
Inverse word labels: ['of' 'V' 'with' 'the']
[Predicted_val - Actual_val]: [ 0.07523686 -0.40865264 -0.2297339   0.02795204]
feature_count: 299
Inverse word labels: ['the' 'shape' 'of' 'diagonal']
[Predicted_val - Actual_val]: [-0.05845501 -0.17453904  0.06739492 -0.50691336]
feature_count: 299
Inverse word labels: ['form' 'of' 'deer' 'horns']
[Predicted_val - Actual_val]: [ 0.19767215 -0.24297475 -0.51945323 -0.51736909]
feature_count: 299
Inverse word labels: ['is' 'also' 'a' 'pink']
[Predicted_val - Actual_val]: [-0.02254706 -0.30031071 -0.01155903  0.48545938]
feature_count: 299
Inverse word labels: ['the' 'shape' 'of' 'horns']
[Predicted_val - Actual_val]: [-0.03986682 -0.0467649  -0.2112411

Inverse word labels: ['light' 'blue' 'but' 'it']
[Predicted_val - Actual_val]: [-0.33972354 -0.23484651 -0.18508795 -0.01312552]
feature_count: 299
Inverse word labels: ['with' 'triangles' 'on' 'a']
[Predicted_val - Actual_val]: [-0.25988296 -0.07978886 -0.06595923  0.01139615]
feature_count: 299
Inverse word labels: ['a' 'square' 'on' 'the']
[Predicted_val - Actual_val]: [-0.01895172 -0.24871073 -0.14431046 -0.04042604]
feature_count: 299
Inverse word labels: ['the' 'same' 'shape' 'you']
[Predicted_val - Actual_val]: [-0.04192701  0.23594443 -0.30945006 -0.08630074]
feature_count: 299
Inverse word labels: ['shape' 'of' 'triangle' 'and']
[Predicted_val - Actual_val]: [ 0.07304154 -0.10384683 -0.44323054 -0.27365427]
feature_count: 299
Inverse word labels: ['four' 'part' 'it' "'s"]
[Predicted_val - Actual_val]: [-0.11051398  0.38951179 -0.01304228  0.0033299 ]
feature_count: 299
Inverse word labels: ['the' 'bottom' 'instead' 'of']
[Predicted_val - Actual_val]: [-0.04404653 -0.28519349  